In [1]:
!pip install --q crewai
!pip install --q langchain_groq
!pip install --q langchain_chatOpenAI
!pip install --q 'crewai[tools]'
!pip install --q dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 794.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.8/206.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.6/327.6 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.5/339.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [3]:
import os
import yaml
from crewai import Agent, Task, Crew, Process
from crewai_tools import PDFSearchTool
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [4]:
# Carregar as variáveis de ambiente do arquivo .env
load_dotenv()

# Configuração das chaves de API (substitua com suas chaves reais)
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

# Caminho da pasta contendo os PDFs
pdf_folder = r"PDFs"

#Chave

#SOLICITAÇÕES

In [9]:
solicitacoes = """
\n<solicitacoes>\n
1 - OBJETIVOS - Identificação dos Objetivos: Realize uma análise cuidadosa do conteúdo do trabalho para extrair os objetivos principais. Resuma esses objetivos em um parágrafo claro e conciso, capturando a essência das metas e intenções do estudo.\n
2 - GAP - Identificação do GAP: Analise o conteúdo do trabalho para identificar o GAP científico que ele aborda, mesmo que não esteja explicitamente mencionado. Formule um parágrafo conciso, focando em destacar a questão central que o estudo procura resolver ou elucidar.\n
3 - METODOLOGIA - Extração Detalhada da Metodologia do Trabalho: Identificação e Descrição da Metodologia: Proceda com uma análise minuciosa do trabalho para identificar a metodologia utilizada. Detalhe cada aspecto da metodologia, incluindo o desenho do estudo, as técnicas e ferramentas empregadas, os procedimentos de coleta e análise de dados, os passos do método e quaisquer metodologias específicas ou inovadoras adotadas. Formule uma descrição compreensiva em texto corrido, limitando-se a um máximo de 250 palavras para manter a concisão sem sacrificar detalhes importantes.\n
4 - DATASET - Identifique os datasets usados no trabalho. Descreva-os brevemente em texto corrido, limitando-se a 40 palavras. Quero somente o nome dos dataset na mesma linha e separados por virgula. Se o dataset foi criado pelos autores escreve "OWN DATASET"\n
5 - RESULTADOS - Escreva em um parágrafo os resultados obitidos estudo dando enfase a dados quantitativos, quero dados numéricos explicitamente. Nesse paragrafo também dê enfase a comparação ao melhor trabalho anterior em relação ao trabalho proposto. Não use superlativos. Deixe o tom neuro e científico.\n
6 - LIMITAÇÕES - Produza um texto parafraseado das limitações do trabalho.\n
7 - CONCLUSÃO - Resuma as conclusões dos autores em relação ao trabalho.\n
8 - FUTURO - Extraia as Recomendações para Pesquisa Futura: Aponte recomendações para futuras investigações baseadas nas conclusões do artigo.\n
9 - AVALIAÇÃO - Faça uma avalição crítica ao trabalho. Não seja generalista faça uma crítica aprofundada.\n
</solicitacoes>\n
"""

#CONTROLES

In [10]:
controles = """
\n<controle>\n
NÍVEIS DE CONTROLE:\n
1. Entonação:Formal Científico.\n
2. Foco de Tópico: Você deve reponder sempre com alto foco no texto do artigo científico.\n
3. Língua: Responda sempre em Português do Brasil como os Brasileiros costumam escrever textos científicos aderindo aos padrões de redação científica do país a não ser o que será especificado para não traduzir.\n
4. Controle de Sentimento: Neutro e científico. Evite superlativos como: inovador, revolucionário e etc\n
5. Nível Originalidade: 10, onde 1 é pouco original e 10 é muito original. Em hipótese alguma copie frases do texto original.\n
6. Nível de Abstração: 1, onde 1 é muito concreto e real e 10 é muito abstrado e irreal.\n
7. Tempo Verbal: Escreva no passado.\n
</controle>\n
"""

#Restrições

In [11]:
restricoes = """
\n<restricoes>\n
O QUE NÃO DEVE SER TRADUZIDO DO INGLÊS PARA PORTUGUÊS:\n
1. Termos técnicos em inglês amplamente aceitos e usado nos textos em português. \n
2. Nome de algoritmos de machine learning.\n
3. Métricas usadas no trabalho.\n
4. Nome dos datasets.\n
5. Não envolva o retorno do YAML com ```yaml. \n
6. Não coloque ``` nem ´´´ no texto de retorno. \n
</restricoes>\n
"""

#Template de saída

In [12]:
# Modelo de referência para o YAML
template = """
\n<template>\n
ARTIGO:\n
  - ARQUIVO: "nome do arquivo.pdf"\n
  - OBJETIVOS: "Objetivo geral e específicos"\n
  - GAP: "Gap científico"\n
  - METODOLOGIA: "Metodologia"\n
  - DATASET: "Datasets utilizados"\n
  - RESULTADOS: "Resultados do artigo"\n
  - LIMITAÇÕES: "Limitações do artigo científico"\n
  - CONCLUSÃO: "Conclusões"\n
  - AVALIAÇÃO: "Análise do artigo"\n
</template>\n
"""

In [13]:
def create_agent_leitor(llm, tool):

    return Agent(
        role='PDF Reader',
        goal="Ler PDFs e extrair informações específicas conforme definido nas solicitações em <solicitacoes>. "
             "Gerar um YAML de acordo com o modelo especificado em <template>. {solicitacoes} {template}.",
        backstory="Você é um especialista em leitura e análise de artigos científicos. "
                  "Sua missão é extrair informações cruciais, compreendendo o contexto semântico completo dos artigos. "
                  "Sua função é fundamental para avaliar a relevância dos artigos analisados. "
                  "Ao responder às solicitações delimitadas por <solicitacoes></solicitacoes>, "
                  "você deve levar em consideração as definições de controles em <controle></controle> "
                  "e as restrições em <restrições></restrições>. "
                  "{solicitacoes} {template} {restricoes} {controles}",
        tools=[tool],
        verbose=True,
        memory=False,
        llm=llm
    )

#Agente Revisor

In [14]:
def create_agent_revisor(llm):
    return Agent(
        role="Revisor de leitura",

        goal="Leia os dados extraídos pelo Agente Revisor e verifique se um YAML foi produzido "
             "de acordo com o template proposto em <template>, "
             "com os dados solicitados em <solicitacoes>. "
             "Como resultado do seu trabalho, você deve retornar um YAML "
             "revisado no mesmo formato do template proposto. {solicitacoes} {template}",

        backstory="Você é um especialista na revisão de informações em YAML, "
                  "especialmente de resumos de artigos científicos. "
                  "Sua função é garantir que os dados extraídos reflitam "
                  "com precisão as solicitações definidas em <solicitacoes> "
                  "e estejam formatados conforme o template proposto em <template>. "
                  "Sua atenção aos detalhes assegura que os resultados finais "
                  "sejam precisos e conformes às expectativas. {solicitacoes} {template}",
        verbose=True,
        memory=False,
        llm=llm
    )


# Função para criar uma tarefa de leitura de PDF

In [15]:
def leitor_task(agent_leitor):
    return Task(
        description="Leia o PDF e responda em YAML às solicitações definidas em <solicitacoes> "
            "usando o modelo definido em <template>. "
            "{solicitacoes} {template}",
        expected_output="YAML com as respostas às solicitações definidas em "
                        "<solicitacoes>, usando o modelo definido em <template>.",
        agent=agent_leitor
    )

# Função para criar uma tarefa do Revisor

In [16]:
def revisor_task(agent_revisor):
    return Task(
        description=
            "Revise o YAML produzido pelo agente leitor para garantir que ele esteja de acordo com o template definido em <template> "
            "e contenha todas as informações solicitadas em <solicitacoes>. {solicitacoes} {template}"
        ,
        expected_output="YAML revisado que esteja de acordo com o template definido em <template> "
        "e contenha todas as informações solicitadas em <solicitacoes>. {solicitacoes} {template}",
        agent=agent_revisor
    )


In [17]:
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

In [18]:
pdf_files

['Low-light aware framework for human activity recognition via optimized dual stream parallel network.pdf',
 '01 - BP-Net Boundary and perfusion feature guided dual-modality ultrasound video analysis network for fibrous cap integrity assessment.pdf']

# Lista para armazenar os resultados de todos os artigos

In [19]:
all_articles = []

In [20]:
for pdf_file_name in pdf_files:

    #criando a mente dos agentes
    gpt = ChatOpenAI(model="gpt-4-turbo")

    pdf = os.path.join(pdf_folder, pdf_file_name)

    pdf_tool = PDFSearchTool(pdf)

    #LEITOR
    agent_leitor = create_agent_leitor(gpt, pdf_tool)
    task_leitor = leitor_task(agent_leitor)

    #REVISOR
    agent_revisor = create_agent_revisor(gpt)
    task_revisor = revisor_task(agent_revisor)

    crew = Crew(
        agents=[agent_leitor, agent_revisor],
        tasks=[task_leitor, task_revisor],
        process=Process.sequential
    )

    ipt = {
      'solicitacoes': solicitacoes,
      'template': template,
      'restricoes': restricoes,
      'controles': controles
    }

    results = crew.kickoff(inputs = ipt)

    #retirando as marcações de yaml retornadas pelo GPT
    results = results.split("yaml")[1].split("```")[0].strip()

    # Carregar a string YAML em um objeto Python
    article_data = yaml.safe_load(results)

    all_articles.append(article_data)


# Estrutura final para o arquivo YAML
final_output = {'artigos': all_articles}

# Definir o nome do arquivo onde o YAML será salvo
file_name = 'output.yaml'

# Salvar o objeto Python em um arquivo YAML
with open(file_name, 'w') as file:
    yaml.dump(final_output, file, default_flow_style=False, allow_unicode=True)

print(f"Dados salvos em {file_name}")
